In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.4` // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:0.10.9` 

import $ivy.$                                   // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:0.10.9` 

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)


import org.apache.log4j.{Level, Logger}


In [3]:
import org.apache.spark.sql._


import org.apache.spark.sql._


In [4]:
val spark = {
  SparkSession.builder()
    .master("local[*]")
    .config("spark.testing.memory", 471859200)
    .config("spark.executor.instances", "4")
    .config("spark.executor.memory", "2g")
    .getOrCreate()
}

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


spark: SparkSession = org.apache.spark.sql.SparkSession@2c00fedc

In [5]:
val df = spark.read.format("parquet").load("data/train")

: 

In [ ]:
df.select("feedback").show(10)